<a href="https://colab.research.google.com/github/Nablasquarednull/BCH_proyect/blob/main/BCH_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from operator import indexOf
import sympy
from sympy import symbols
from sympy.plotting import plot
sympy.init_printing()
from sympy import*
from sympy.physics.quantum import *
from sympy.physics.quantum import Operator, Commutator, AntiCommutator
from sympy.physics.quantum import Dagger, Commutator
from sympy.physics.quantum.boson import BosonOp
from math import ceil
from sympy.printing.latex import LatexPrinter, print_latex
#-----------------------------
#-----------------------------

def collect_coeff(expresion,A): #factoriza el operador A de una expresión "expresion"
  vt = 0
  for k in range(len(expand(expresion).args)):
    if (expand(expresion).args[k]).coeff(A)!=0:
      vt+= (expand(expresion).args[k].coeff(A))
  return (vt)*A
#--------------------------------------
def collect_coeff2(expresion,A): #factoriza el operador A de una expresión "expresion" NOTA AL DR. RAMOS: MODIFIQUÉ LA EXPRESIÓN FINAL: AHORA REGRESA (vt), antes regresaba (vt)*A
  vt = 0
  for k in range(len(expand(expresion).args)):
    if (expand(expresion).args[k]).coeff(A)!=0:
      vt+= (expand(expresion).args[k].coeff(A))
  return (vt)
#-----------------------------
"""def conmutador(A,B):
  return Commutator(A,B).expand(commutator=True).expand(commutator=True).subs({Commutator(x,p):I, Commutator(p,x):-I,Commutator(p,p):0,Commutator(x,x):0,

                                                                               Commutator(a,a_dag):1,Commutator(a_dag,a):-1,Commutator(a_dag,a_dag):0,Commutator(a,a):0,

                                                                               Commutator(sigmap,sigmam):sigmaz,Commutator(sigmam,sigmap):-sigmaz,
                                                                               Commutator(sigmaz,sigmap):2*sigmap,
                                                                               Commutator(sigmap,sigmaz):-2*sigmap,Commutator(sigmam,sigmaz):2*sigmam,
                                                                               Commutator(sigmaz,sigmam):-2*sigmam,
                                                                               Commutator(sigmap,sigmap):0,Commutator(sigmam,sigmam):0,Commutator(sigmaz,sigmaz):0})
                                                                               """
# Define the commutator substitution dictionary


# Optimized conmutador function
def conmutador(A, B):
    return Commutator(A, B).expand(commutator=True).expand(commutator=True).subs(commutator_subs)

def com_order(A,B,n): #regresa el conmutador anidado [A,[A,..[A,B]] n veces.
    if n<=1:
        return conmutador(A,B)

    else:
        return com_order(A,conmutador(A,B),n-1)
#-----------------------------
class MyLatexPrinter(LatexPrinter):
    """Print derivative of a function of symbols in a shorter form.
    """
    def _print_Derivative(self, expr):
        function, *vars = expr.args
        if not isinstance(type(function), UndefinedFunction) or \
           not all(isinstance(i, Symbol) for i in vars):
            return super()._print_Derivative(expr)

        # If you want the printer to work correctly for nested
        # expressions then use self._print() instead of str() or latex().
        # See the example of nested modulo below in the custom printing
        # method section.
        return "{}_{{{}}}".format(
            self.print(Symbol(function.func.name_)),
                        ''.join(self._print(i) for i in vars))


def Latex(expr):
    """ Most of the printers define their own wrappers for print().
    These wrappers usually take printer settings. Our printer does not have
    any settings.
    """
    print(MyLatexPrinter().doprint(expr))
#------------------------------
def BCH(A, B, alpha, n): # Baker_Campbell_Hausdorff formula
    a=0
    for i in range(1,n+1):
        a += (alpha**i/factorial(i))*com_order(A,B,i)
    return (B + a)
#-------------------------------
def collect_all(expression,operator_list): # itera "len(operator list)" veces la función "collect_coef" y suma todas sus iteraciones.
  a=0
  for i in range(len(operator_list)):
    a += collect_coeff(expression,operator_list[i])
  return a
#-------------------------------
def collect_and_save_all(expression,operator_list):
  coeff_list=[] #lista donde vamos a meter los coeficientes de cada operador
  for z in range(len(operator_list)): #iterador para cada operador
    a = collect_coeff2(expression,operator_list[z])
    coeff_list.append(a)
  return coeff_list
#-------------------------------
def compare_all(coeff_list,operator_list,n): #compara los coeficientes y los reemplaza.
  list_1= coeff_list.copy()
  functions = [cos, sin , exp] #lista de funciones para comparar
  for z in range(len(operator_list)):
    for i in functions: #iterador para cada funcion
      for y in (-2,1,0,1,2): #iterador para el orden de las expansiones
        if coeff_list[z] == series(i(omega*t),omega*t,0,n+y).removeO(): #funciones en 'functions' positivas
          list_1[z] = i(omega*t)
          break
        elif coeff_list[z] == series(-i(omega*t),omega*t,0,n+y).removeO():# funciones en 'functions' negativas
          list_1[z] = -i(omega*t)
          break
        elif coeff_list[z] == fraction(series(i(omega*t)/omega,omega,0,n+y).removeO())[0]: #funciones en 'functions' cardinales positivas
          list_1[z] = i(omega*t)/omega
          break
        elif coeff_list[z] == fraction(series(-i(omega*t)/omega,omega,0,n+y).removeO())[0]:#funciones en 'functions' cardinales negativas
          list_1[z] = -i(omega*t)/omega
          break
        elif coeff_list[z] == series(i(omega*t)*omega,omega,0,n+y).removeO(): #funciones en 'functions' cardinales positivas
          list_1[z] = i(omega*t)*omega
          break
        elif coeff_list[z] == series(-i(omega*t)*omega,omega,0,n+y).removeO():#funciones en 'functions' cardinales negativas
          list_1[z] = -i(omega*t)*omega
          break

  return list_1

#-------------------------------
def sum_collect(coef_list,operator_list):
  a=0
  for i in range(len(operator_list)):
    a += coef_list[i]*(operator_list[i])
  return a
#-------------------------------

def differentiate(T,t):
  return T.diff(t)
 #-------------------------------
def diff_term(T,t):
  return - I*Dagger(T)*T.diff(t)
#-------------------------------




x     = Operator(r'\hat{x}')
p     = Operator(r'\hat{p}')
omega = symbols(r'\omega', real = True)
alpha = symbols(r'\alpha')
t     = symbols(r't', real = True)
H     = Rational(1,2)*(p**2+omega**2*x**2)
l =[x,p]
sigmap = Operator(r'\hat{\sigma}_+')
sigmam = Operator(r'\hat{\sigma}_-')
sigmaz = Operator(r'\hat{\sigma}_z')

H_2 = sigmaz + sigmap + sigmam
l_2 = [sigmaz,sigmap,sigmam]
#---------------------------------------
a_dag = Operator(r'\hat{a}^{\dagger}')
a = Operator(r'\hat{a}')
H_3 = omega*(a_dag*a+1/2)
l_3 = [a,a_dag]
#---------------------------------------
commutator_subs = {
    Commutator(x, p): I,
    Commutator(p, x): -I,
    Commutator(p, p): 0,
    Commutator(x, x): 0,

    Commutator(a, a_dag): 1,
    Commutator(a_dag, a): -1,
    Commutator(a_dag, a_dag): 0,
    Commutator(a, a): 0,

    Commutator(sigmap, sigmam): sigmaz,
    Commutator(sigmam, sigmap): -sigmaz,

    Commutator(sigmaz, sigmap): 2 * sigmap,
    Commutator(sigmap, sigmaz): -2 * sigmap,
    Commutator(sigmam, sigmaz): 2 * sigmam,
    Commutator(sigmaz, sigmam): -2 * sigmam,

    Commutator(sigmap, sigmap): 0,
    Commutator(sigmam, sigmam): 0,
    Commutator(sigmaz, sigmaz): 0,
}
#--------------------------------------------
orden = 8

In [ ]:
BCH_X = BCH(H,x,alpha,orden).subs({alpha:I*t})
BCH_X

      8  8                 6  7                 6  6                 4  5                 4  4     ↪
\omega ⋅t ⋅\hat{x}   \omega ⋅t ⋅\hat{p}   \omega ⋅t ⋅\hat{x}   \omega ⋅t ⋅\hat{p}   \omega ⋅t ⋅\ha ↪
────────────────── - ────────────────── - ────────────────── + ────────────────── + ────────────── ↪
      40320                 5040                 720                  120                   24     ↪

↪              2  3                 2  2                              
↪ t{x}   \omega ⋅t ⋅\hat{p}   \omega ⋅t ⋅\hat{x}                      
↪ ──── - ────────────────── - ────────────────── + t⋅\hat{p} + \hat{x}
↪                6                    2                               

In [ ]:
BCH_X_1 = collect_all(BCH_X,l)
BCH_X_1

⎛        6  7         4  5         2  3    ⎞           ⎛      8  8         6  6         4  4       ↪
⎜  \omega ⋅t    \omega ⋅t    \omega ⋅t     ⎟           ⎜\omega ⋅t    \omega ⋅t    \omega ⋅t    \om ↪
⎜- ────────── + ────────── - ────────── + t⎟⋅\hat{p} + ⎜────────── - ────────── + ────────── - ─── ↪
⎝     5040         120           6         ⎠           ⎝  40320         720           24           ↪

↪    2  2    ⎞        
↪ ega ⋅t     ⎟        
↪ ─────── + 1⎟⋅\hat{x}
↪  2         ⎠        

In [ ]:
BCH_X_2 = collect_and_save_all(BCH_X,l)
BCH_X_2

⎡      8  8         6  6         4  4         2  2              6  7         4  5         2  3    ⎤
⎢\omega ⋅t    \omega ⋅t    \omega ⋅t    \omega ⋅t         \omega ⋅t    \omega ⋅t    \omega ⋅t     ⎥
⎢────────── - ────────── + ────────── - ────────── + 1, - ────────── + ────────── - ────────── + t⎥
⎣  40320         720           24           2                5040         120           6         ⎦

In [ ]:
BCH_X_3 = compare_all(BCH_X_2,l,orden)
BCH_X_3

⎡               sin(\omega⋅t)⎤
⎢cos(\omega⋅t), ─────────────⎥
⎣                  \omega    ⎦

In [ ]:
BCH_X_4 = sum_collect(BCH_X_3,l)
BCH_X_4

                        sin(\omega⋅t)⋅\hat{p}
cos(\omega⋅t)⋅\hat{x} + ─────────────────────
                               \omega        

In [ ]:
BCH_P = BCH(H,p,alpha,orden).subs({alpha:I*t})
BCH_P

        10  9                 8  8                 8  7                 6  6                 6  5  ↪
  \omega  ⋅t ⋅\hat{x}   \omega ⋅t ⋅\hat{p}   \omega ⋅t ⋅\hat{x}   \omega ⋅t ⋅\hat{p}   \omega ⋅t ⋅ ↪
- ─────────────────── + ────────────────── + ────────────────── - ────────────────── - ─────────── ↪
        362880                40320                 5040                 720                  120  ↪

↪                 4  4                 4  3                 2  2                                   ↪
↪ \hat{x}   \omega ⋅t ⋅\hat{p}   \omega ⋅t ⋅\hat{x}   \omega ⋅t ⋅\hat{p}         2                 ↪
↪ ─────── + ────────────────── + ────────────────── - ────────────────── - \omega ⋅t⋅\hat{x} + \ha ↪
↪                   24                   6                    2                                    ↪

↪     
↪     
↪ t{p}
↪     

In [ ]:
BCH_P_1 = collect_all(BCH_P,l)
BCH_P_1

⎛      8  8         6  6         4  4         2  2    ⎞           ⎛        10  9         8  7      ↪
⎜\omega ⋅t    \omega ⋅t    \omega ⋅t    \omega ⋅t     ⎟           ⎜  \omega  ⋅t    \omega ⋅t    \o ↪
⎜────────── - ────────── + ────────── - ────────── + 1⎟⋅\hat{p} + ⎜- ─────────── + ────────── - ── ↪
⎝  40320         720           24           2         ⎠           ⎝    362880         5040         ↪

↪     6  5         4  3            ⎞        
↪ mega ⋅t    \omega ⋅t          2  ⎟        
↪ ──────── + ────────── - \omega ⋅t⎟⋅\hat{x}
↪  120           6                 ⎠        

In [ ]:
BCH_P_2 = collect_and_save_all(BCH_P,l)
BCH_P_2

⎡        10  9         8  7         6  5         4  3                    8  8         6  6         ↪
⎢  \omega  ⋅t    \omega ⋅t    \omega ⋅t    \omega ⋅t          2    \omega ⋅t    \omega ⋅t    \omeg ↪
⎢- ─────────── + ────────── - ────────── + ────────── - \omega ⋅t, ────────── - ────────── + ───── ↪
⎣    362880         5040         120           6                     40320         720           2 ↪

↪  4  4         2  2    ⎤
↪ a ⋅t    \omega ⋅t     ⎥
↪ ───── - ────────── + 1⎥
↪ 4           2         ⎦

In [ ]:
BCH_P_3 = compare_all(BCH_P_2,l,orden)
BCH_P_3

[-\omega⋅sin(\omega⋅t), cos(\omega⋅t)]

In [ ]:
BCH_P_4 = sum_collect(BCH_P_3,l)
BCH_P_4

-\omega⋅sin(\omega⋅t)⋅\hat{x} + cos(\omega⋅t)⋅\hat{p}

In [ ]:
BCH_sigma = BCH(H_2,sigmaz,alpha,7).subs({alpha:I*t})
BCH_sigma

     7                                               6                                             ↪
  ⅈ⋅t ⋅(-1024⋅\hat{\sigma}₊ + 1024⋅\hat{\sigma}₋)   t ⋅(-256⋅\hat{\sigma}₊ - 256⋅\hat{\sigma}₋ + 2 ↪
- ─────────────────────────────────────────────── - ────────────────────────────────────────────── ↪
                       5040                                                       720              ↪

↪                         5                                             4                          ↪
↪ 56⋅\hat{\sigma}_z)   ⅈ⋅t ⋅(-128⋅\hat{\sigma}₊ + 128⋅\hat{\sigma}₋)   t ⋅(-32⋅\hat{\sigma}₊ - 32⋅ ↪
↪ ────────────────── + ───────────────────────────────────────────── + ─────────────────────────── ↪
↪                                           120                                                    ↪

↪                                         3                                           2            ↪
↪ \hat{\sigma}₋ + 32⋅\hat{\sigma}_z)   ⅈ⋅t ⋅(-16⋅\hat{\sigma}₊ + 16⋅\hat{\sigma}₋)   t ⋅(

In [ ]:
BCH_sigma_1 = collect_all(BCH_sigma,l_2)
BCH_sigma_1

⎛      6      4           ⎞                  ⎛        7       6         5      4        3          ↪
⎜  16⋅t    4⋅t       2    ⎟                  ⎜  64⋅ⅈ⋅t    16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2   ↪
⎜- ───── + ──── - 2⋅t  + 1⎟⋅\hat{\sigma}_z + ⎜- ─────── + ───── + ─────── - ──── - ────── + 2⋅t  + ↪
⎝   45      3             ⎠                  ⎝    315      45       15       3       3             ↪

↪       ⎞                 ⎛      7       6         5      4        3               ⎞              
↪       ⎟                 ⎜64⋅ⅈ⋅t    16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2        ⎟              
↪  2⋅ⅈ⋅t⎟⋅\hat{\sigma}₋ + ⎜─────── + ───── - ─────── - ──── + ────── + 2⋅t  - 2⋅ⅈ⋅t⎟⋅\hat{\sigma}₊
↪       ⎠                 ⎝  315      45       15       3       3                  ⎠              

In [ ]:
(I*series(sin(t)**2,t,0,8).removeO()).expand()

     6      4       
2⋅ⅈ⋅t    ⅈ⋅t       2
────── - ──── + ⅈ⋅t 
  45      3         

In [ ]:
BCH_sigma_2 = collect_and_save_all(BCH_sigma,l_2)
BCH_sigma_2

⎡      6      4                   7       6         5      4        3                         7    ↪
⎢  16⋅t    4⋅t       2      64⋅ⅈ⋅t    16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2            64⋅ⅈ⋅t     ↪
⎢- ───── + ──── - 2⋅t  + 1, ─────── + ───── - ─────── - ──── + ────── + 2⋅t  - 2⋅ⅈ⋅t, - ─────── +  ↪
⎣   45      3                 315      45       15       3       3                        315      ↪

↪     6         5      4        3               ⎤
↪ 16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2        ⎥
↪ ───── + ─────── - ──── - ────── + 2⋅t  + 2⋅ⅈ⋅t⎥
↪  45       15       3       3                  ⎦

In [ ]:
BCH_sigma_3 = compare_all(BCH_sigma_2,l_2,7)
BCH_sigma_3

⎡      6      4                   7       6         5      4        3                         7    ↪
⎢  16⋅t    4⋅t       2      64⋅ⅈ⋅t    16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2            64⋅ⅈ⋅t     ↪
⎢- ───── + ──── - 2⋅t  + 1, ─────── + ───── - ─────── - ──── + ────── + 2⋅t  - 2⋅ⅈ⋅t, - ─────── +  ↪
⎣   45      3                 315      45       15       3       3                        315      ↪

↪     6         5      4        3               ⎤
↪ 16⋅t    16⋅ⅈ⋅t    4⋅t    8⋅ⅈ⋅t       2        ⎥
↪ ───── + ─────── - ──── - ────── + 2⋅t  + 2⋅ⅈ⋅t⎥
↪  45       15       3       3                  ⎦

In [ ]:
BCH_A = BCH(H_3,a_dag,alpha,7).subs({alpha:I*t})
BCH_A

          7  7                            6  6                              5  5                   ↪
  ⅈ⋅\omega ⋅t ⋅\hat{a}__{\dagger}   \omega ⋅t ⋅\hat{a}__{\dagger}   ⅈ⋅\omega ⋅t ⋅\hat{a}__{\dagger ↪
- ─────────────────────────────── - ───────────────────────────── + ────────────────────────────── ↪
               5040                              720                              120              ↪

↪           4  4                              3  3                            2  2                 ↪
↪ }   \omega ⋅t ⋅\hat{a}__{\dagger}   ⅈ⋅\omega ⋅t ⋅\hat{a}__{\dagger}   \omega ⋅t ⋅\hat{a}__{\dagg ↪
↪ ─ + ───────────────────────────── - ─────────────────────────────── - ────────────────────────── ↪
↪                  24                                6                                2            ↪

↪                                                         
↪ er}                                                     
↪ ─── + ⅈ⋅\omega⋅t⋅\hat{a}__{\dagger} + \hat{a}__{\dagger}
↪            

In [ ]:
BCH_A_1 = collect_all(BCH_A,l_3)
BCH_A_1

⎛          7  7         6  6           5  5         4  4           3  3         2  2               ↪
⎜  ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t                ↪
⎜- ──────────── - ────────── + ──────────── + ────────── - ──────────── - ────────── + ⅈ⋅\omega⋅t  ↪
⎝      5040          720           120            24            6             2                    ↪

↪    ⎞                   
↪    ⎟                   
↪ + 1⎟⋅\hat{a}__{\dagger}
↪    ⎠                   

In [ ]:
BCH_A_2 = collect_and_save_all(BCH_A,l_3)
BCH_A_2

⎡             7  7         6  6           5  5         4  4           3  3         2  2            ↪
⎢     ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t             ↪
⎢0, - ──────────── - ────────── + ──────────── + ────────── - ──────────── - ────────── + ⅈ⋅\omega ↪
⎣         5040          720           120            24            6             2                 ↪

↪       ⎤
↪       ⎥
↪ ⋅t + 1⎥
↪       ⎦

In [ ]:
BCH_A_3 = compare_all(BCH_A_2,l_3,7)
BCH_A_3

⎡             7  7         6  6           5  5         4  4           3  3         2  2            ↪
⎢     ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t    ⅈ⋅\omega ⋅t    \omega ⋅t             ↪
⎢0, - ──────────── - ────────── + ──────────── + ────────── - ──────────── - ────────── + ⅈ⋅\omega ↪
⎣         5040          720           120            24            6             2                 ↪

↪       ⎤
↪       ⎥
↪ ⋅t + 1⎥
↪       ⎦

In [ ]:
Latex(series(exp(-I*omega*t),t,0,8).removeO())

\frac{i \omega^{7} t^{7}}{5040} - \frac{\omega^{6} t^{6}}{720} - \frac{i \omega^{5} t^{5}}{120} + \frac{\omega^{4} t^{4}}{24} + \frac{i \omega^{3} t^{3}}{6} - \frac{\omega^{2} t^{2}}{2} - i \omega t + 1



$
\frac{i \omega^{7} t^{7}}{5040} - \frac{\omega^{6} t^{6}}{720} - \frac{i \omega^{5} t^{5}}{120} + \frac{\omega^{4} t^{4}}{24} + \frac{i \omega^{3} t^{3}}{6} - \frac{\omega^{2} t^{2}}{2} - i \omega t + 1
$
